# Tahmine Dayalı Değer Atama Yöntemleri

### KNN & Random Forests & EM

önceki bölümlerde eksik değerleri ya sildik ya da ortalama medyan gibi basit değerlerle doldurduk bunların ötesinde daha ileri seviyede makine öğrenmesi algoritmalarını kullanarak eksik değerleri dolduralım.

elimizdeki eksik değere sahip olan değişken diğer değişkenlerce modellenip buna göre değerleri doldurulmuş olur.

makine öğrenmesi yöntemleriyle eksik değerlerin nasıl doldurabileceğimizi öğreneceğiz.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno

In [2]:
# titanic veri setini çektim.
df = sns.load_dataset("titanic")

In [3]:
# sadece sayısal değişkenleri aldım
df = df.select_dtypes(include=["float64","int64"])

In [4]:
df.head()

,survived,pclass,age,sibsp,parch,fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [5]:
# her değişkenden kaç tane eksik onu gösterdim
df.isnull().sum()

survived      0
pclass        0
age         177
sibsp         0
parch         0
fare          0
dtype: int64

age'de 177 adet eksik değer var. şimdi bunu tahmine dayalı yöntemleri kullanarak nasıl doldurabileceğimizi öğreneceğiz. 

In [6]:
pip install ycimpute

Note: you may need to restart the kernel to use updated packages.


## KNN

KNN(K-En Yakın Komşu) yöntemiyle -bir makine öğrenmesi algoritması- bir doldurma işlemi gerçekleştireceğiz.

In [7]:
from ycimpute.imputer import knnimput

değişkenlerin ismini tutmamız gerekiyor. çünkü bu atama işlemini yapabilmek için KNN input fonksiyonu bizden bir numpy array'ı bekliyor.

dolayısıyla öncelikle DataFrame'in isimlerini bir yerde saklayıp, daha sonra veri setini numpy array'ine çevireceğiz atama işlemlerinden sonra tekrar isimlendirme işlemlerini gerçekleştireceğiz.

In [8]:
var_names = list(df)
var_names

['survived', 'pclass', 'age', 'sibsp', 'parch', 'fare']

In [10]:
# DataFrame'i numpy array'ine çevirelim
n_df = np.array(df)
n_df[0:10] # DataFrame, numpy array'ine çevrildi.

array([[ 0.    ,  3.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  1.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 1.    ,  3.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       [ 1.    ,  1.    , 35.    ,  1.    ,  0.    , 53.1   ],
       [ 0.    ,  3.    , 35.    ,  0.    ,  0.    ,  8.05  ],
       [ 0.    ,  3.    ,     nan,  0.    ,  0.    ,  8.4583],
       [ 0.    ,  1.    , 54.    ,  0.    ,  0.    , 51.8625],
       [ 0.    ,  3.    ,  2.    ,  3.    ,  1.    , 21.075 ],
       [ 1.    ,  3.    , 27.    ,  0.    ,  2.    , 11.1333],
       [ 1.    ,  2.    , 14.    ,  1.    ,  0.    , 30.0708]])

In [11]:
n_df.shape # gözlem birimi sayısı, değişken sayısı

(891, 6)

In [12]:
# k=4: komşuluk sayısını ifade eder.
# complete(n_df): n_df'i doldur. (numpy array nesnesi)
dff = knnimput.KNN(k = 4).complete(n_df)
# eksik olan değerleri doldurur

Imputing row 1/891 with 0 missing, elapsed time: 0.112
Imputing row 101/891 with 0 missing, elapsed time: 0.114
Imputing row 201/891 with 0 missing, elapsed time: 0.115
Imputing row 301/891 with 1 missing, elapsed time: 0.116
Imputing row 401/891 with 0 missing, elapsed time: 0.116
Imputing row 501/891 with 0 missing, elapsed time: 0.117
Imputing row 601/891 with 0 missing, elapsed time: 0.118
Imputing row 701/891 with 0 missing, elapsed time: 0.118
Imputing row 801/891 with 0 missing, elapsed time: 0.119


In [13]:
type(dff)

numpy.ndarray

In [16]:
# dff, bir numpy array'i olduğundan pandas DataFrame'ine çevirmeliyiz.
# var_names: değişkenlerin isimleriydi
# değişkenin isimlerini dff(numpy array)'e isimlendirme yaparak ekleyerek bunun üzerinden 
# bir DataFrame oluşturuyoruz
dff = pd.DataFrame(dff, columns = var_names)
dff.head(10)

,survived,pclass,age,sibsp,parch,fare
0,0.0,3.0,22.000,1.0,0.0,7.2500
1,1.0,1.0,38.000,1.0,0.0,71.2833
2,1.0,3.0,26.000,0.0,0.0,7.9250
3,1.0,1.0,35.000,1.0,0.0,53.1000
4,0.0,3.0,35.000,0.0,0.0,8.0500
5,0.0,3.0,39.375,0.0,0.0,8.4583
6,0.0,1.0,54.000,0.0,0.0,51.8625
7,0.0,3.0,2.000,3.0,1.0,21.0750
8,1.0,3.0,27.000,0.0,2.0,11.1333
9,1.0,2.0,14.000,1.0,0.0,30.0708


In [17]:
type(dff) # DataFrame

pandas.core.frame.DataFrame

In [18]:
# oluşturduğumuz pandas DataFrame üzerinden eksik değerleri sorgulayalım
dff.isnull().sum()

survived    0
pclass      0
age         0
sibsp       0
parch       0
fare        0
dtype: int64

## Random Forests (farklı py dosyasında)

In [24]:
df = sns.load_dataset("titanic")

In [25]:
df = df.select_dtypes(include = ["float64","int64"])

In [26]:
# DataFrame içerisindeki eksiklikler
df.isnull().sum()

survived      0
pclass        0
age         177
sibsp         0
parch         0
fare          0
dtype: int64

In [27]:
# değişkenlerin isimlerini aldım.
var_names = list(df)

In [28]:
# numpy array'ine dönüştürdüm.
n_df = np.array(df)

In [29]:
from ycimpute.imputer import iterforest

In [30]:
# n_df'in içerisindeki eksik değerleri doldur: complete(n_df)
dff = iterforest.IterImput().complete(n_df)

AttributeError: module 'ycimpute.imputer.iterforest' has no attribute 'IterImput'

In [ ]:
# dff'i, var_names(değişkenlerin isimleri) ile DataFrame'e çevirdik
dff = pd.DataFrame(dff, columns = var_names)
dff.head(10)

In [ ]:
# eksik değerleri sorgulayalım
dff.isnull().sum()

# EM Algoritmasıyla

In [31]:
df = sns.load_dataset("titanic")

In [32]:
df = df.select_dtypes(include = ["float64","int64"])

In [33]:
from ycimpute.imputer import EM

In [34]:
var_names = list(df)

In [35]:
n_df = np.array(df)
n_df[:10]

array([[ 0.    ,  3.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  1.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 1.    ,  3.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       [ 1.    ,  1.    , 35.    ,  1.    ,  0.    , 53.1   ],
       [ 0.    ,  3.    , 35.    ,  0.    ,  0.    ,  8.05  ],
       [ 0.    ,  3.    ,     nan,  0.    ,  0.    ,  8.4583],
       [ 0.    ,  1.    , 54.    ,  0.    ,  0.    , 51.8625],
       [ 0.    ,  3.    ,  2.    ,  3.    ,  1.    , 21.075 ],
       [ 1.    ,  3.    , 27.    ,  0.    ,  2.    , 11.1333],
       [ 1.    ,  2.    , 14.    ,  1.    ,  0.    , 30.0708]])

In [36]:
# n_df'deki eksik olan değerleri doldur.
dff = EM().complete(n_df)

In [37]:
dff = pd.DataFrame(dff, columns=var_names)
dff.head(7)

,survived,pclass,age,sibsp,parch,fare
0,0.0,3.0,22.0,1.0,0.0,7.2500
1,1.0,1.0,38.0,1.0,0.0,71.2833
2,1.0,3.0,26.0,0.0,0.0,7.9250
3,1.0,1.0,35.0,1.0,0.0,53.1000
4,0.0,3.0,35.0,0.0,0.0,8.0500
5,0.0,3.0,0.0,0.0,0.0,8.4583
6,0.0,1.0,54.0,0.0,0.0,51.8625


In [38]:
dff.isnull().sum()

survived    0
pclass      0
age         0
sibsp       0
parch       0
fare        0
dtype: int64

eksik gözlemleri doldurmadan önce, öncelikle bir yapısal problem olup olmadığına bakılır. yani, rastgelelik problemi var mı? diğer bir ifadeyle bazı değişkenlerdeki eksiklikler acaba diğer değişkenlere bağlı olarak mı gerçekleşiyor? buna mutlaka dikkat edilmelidir.